<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Valuation_Gap_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-11-19 00:12:35


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypf = mypf[mypf['Criteria'].isin(['V40','V40N'])]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','PGHH','SIEMENS', 'GILLETTE', 'NESTLEIND', 'SANOFI']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100[:]
#stock_ids = ['HDFCBANK', 'COLPAL']
print('Stocks to be scanned:',len(stock_ids))

Stocks to be scanned: 73


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids:


    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"

    stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL' , 'IEX', 'ROSSARI', 'VALIANTORG', 'VINATIORGA', 'TATAELXSI',
                  'AMARAJABAT', 'EQUITAS']

    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list
    #print(actual_link)
    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list)

    if n < 100:
        actual_link = link_blueprint + stock_name.upper()
        test_list = get_list(actual_link)
        #print(len(test_list))

    #print(test_list)
    if 'TTM' not in test_list:
        end_year = 'Mar 2023'
    else:
        end_year = 'TTM'

        df_yoy = pd.DataFrame()
        #print(test_list)
        s = test_list.index(end_year) +1 - test_list.index('Mar 2017')

        df_yoy['Year'] = test_list[test_list.index('Mar 2017'):test_list.index(end_year)+1]
        df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
        df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

        df_yoy.set_index('Year', inplace=True)
        df_yoy = df_yoy.T
        df_yoy['Stock'] = stock_name.upper() + '.NS'
        #df_yoy
        df_latest_analysis = pd.concat([df_latest_analysis, df_yoy])

df_latest_analysis.index.names = ['Type']
df_latest_analysis.reset_index(inplace=True)
df_latest_analysis.rename(columns={'TTM': 'Nov 2023'}, inplace=True)
#df_latest_analysis.drop('TTM', axis=1, inplace=True)
cols = [ 'Mar 2017','Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', 'Nov 2023']
for col in cols:
    df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

3MINDIA
5PAISA
ABBOTINDIA
AKZOINDIA
ANGELONE
ASIANPAINT
ASTRAZEN
AXISBANK
BAJAJ-AUTO
BAJAJFINSV
BAJAJHLDNG
BAJFINANCE
BATAINDIA
BAYERCROP
BERGEPAINT
BOSCHLTD
CAPLIPOINT
CERA
COLPAL
DABUR
DIXON
EICHERMOT
EQUITASBNK
ERIS
FINCABLES
FINEORG
GLAXO
HAVELLS
HCLTECH
HDFCAMC
HDFCBANK
HDFCLIFE
HINDUNILVR
HONAUT
ICICIBANK
ICICIGI
ICICIPRULI
INDIGOPNTS
INFY
ISEC
ITC
JCHAC
KANSAINER
KOTAKBANK
LALPATHLAB
LUXIND
MARICO
MCDOWELL-N
MCX
MOTILALOFS
NAM-INDIA
OFSS
PAGEIND
PFIZER
PIDILITIND
POLYCAB
RADICO
RAJESHEXPO
RELAXO
RELIANCE
SFL
SIS
SUNTV
SYMPHONY
TATAELXSI
TCS
TEAMLEASE
TITAN
TTKPRESTIG
UJJIVANSFB
VINATIORGA
VIPIND
WHIRLPOOL


In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

stock_df_final = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  short_window = 20
  mid_window = 50
  long_window = 200
  moving_avg = 'SMA'

  # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
  stock_df = yfin.Ticker(stock_symbol).history(period='7y', interval='1mo')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
  stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
  stock_df.dropna(axis = 0, inplace = True) # remove any null rows

  stock_df['Stock'] = stock_symbol

  stock_df['Close'] = round(stock_df['Close'],2)
  stock_df['Mon'] = stock_df.index.month_name().str.slice(stop=3)
  stock_df['Year'] = stock_df.index.strftime('%Y')
  stock_df['Year'] = stock_df['Mon'] + ' ' + stock_df['Year']
  cols = [ 'Year', 'Close']
  stock_df = stock_df[cols]
  stock_df = stock_df[(stock_df['Year'].str.contains("Mar")) | (stock_df['Year'].str.contains("Nov 2023"))]
  stock_df = stock_df.set_index('Year')
  stock_df = stock_df.T
  stock_df['Stock'] = stock_symbol
  stock_df.index.names = ['Type']
  stock_df = stock_df.reset_index()
  stock_df_final = pd.concat([stock_df_final, stock_df])

stock_df_final

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Nov 2023,Stock
0,Close,11123.66,18667.90,23329.87,18105.45,29268.68,18941.44,22882.53,30357.40,3MINDIA.NS
0,Close,NaN,203.15,144.82,113.15,249.70,344.70,283.00,451.55,5PAISA.NS
0,Close,4387.83,5202.22,6973.80,14775.45,14573.12,17372.22,21929.65,23625.50,ABBOTINDIA.NS
0,Close,1673.61,1586.23,1617.28,2013.28,2122.52,1826.17,2266.63,2481.10,AKZOINDIA.NS
0,Close,NaN,NaN,NaN,NaN,268.62,1466.12,1136.93,2918.95,ANGELONE.NS
...,...,...,...,...,...,...,...,...,...,...
0,Close,471.36,497.50,706.14,474.70,712.50,817.36,693.90,786.25,TTKPRESTIG.NS
0,Close,NaN,NaN,NaN,26.22,29.29,14.21,24.25,56.00,UJJIVANSFB.NS
0,Close,366.34,436.41,806.26,762.11,1386.22,1940.29,1801.58,1730.00,VINATIORGA.NS
0,Close,189.78,309.13,472.52,237.62,350.42,737.80,572.70,619.45,VIPIND.NS


In [6]:
df_gap_analysis = df_latest_analysis.append(stock_df_final)
df_gap_analysis['Growth%'] = (df_gap_analysis['Nov 2023'] - df_gap_analysis['Mar 2018'])*100/df_gap_analysis['Mar 2018']
df_gap_analysis['Growth%'] = round(df_gap_analysis['Growth%'],2)
df_gap_analysis

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Nov 2023,Stock,Growth%
0,Sales,2363.00,2556.00,2809.00,2765.00,2420.00,3078.00,3733.00,3871.00,3MINDIA.NS,51.45
1,Profit,369.00,480.00,497.00,320.00,202.00,360.00,561.00,655.00,3MINDIA.NS,36.46
2,Sales,7.00,20.00,61.00,108.00,195.00,298.00,338.00,356.00,5PAISA.NS,1680.00
3,Profit,-16.00,-33.00,-25.00,-10.00,20.00,18.00,58.00,78.00,5PAISA.NS,-336.36
4,Sales,2903.00,3298.00,3679.00,4093.00,4310.00,4913.00,5349.00,5638.00,ABBOTINDIA.NS,70.95
...,...,...,...,...,...,...,...,...,...,...,...
0,Close,471.36,497.50,706.14,474.70,712.50,817.36,693.90,786.25,TTKPRESTIG.NS,58.04
0,Close,NaN,NaN,NaN,26.22,29.29,14.21,24.25,56.00,UJJIVANSFB.NS,NaN
0,Close,366.34,436.41,806.26,762.11,1386.22,1940.29,1801.58,1730.00,VINATIORGA.NS,296.42
0,Close,189.78,309.13,472.52,237.62,350.42,737.80,572.70,619.45,VIPIND.NS,100.38


In [7]:
stock_df_gap = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  #print(stock_symbol)
  tmp_df = df_gap_analysis[df_gap_analysis['Stock'] == stock_symbol]
  gap_perc = round(tmp_df[tmp_df['Type'] == 'Profit']['Growth%'].values[0]  - tmp_df[tmp_df['Type'] == 'Close']['Growth%'].values[0],2)
  #print(gap_perc)
  tmp_df = pd.DataFrame()
  tmp_df['Stock'] = [stock_symbol]
  tmp_df['Gap%'] = [gap_perc]
  stock_df_gap = pd.concat([stock_df_gap, tmp_df])

stock_df_gap.sort_values(['Gap%'], ascending=[False]).head(30)

,Stock,Gap%
0,AXISBANK.NS,3633.28
0,ICICIBANK.NS,165.62
0,BAYERCROP.NS,128.05
0,SIS.NS,93.70
0,MOTILALOFS.NS,93.42
0,KANSAINER.NS,85.00
0,HDFCBANK.NS,83.97
0,KOTAKBANK.NS,78.83
0,SUNTV.NS,52.54
0,BAJFINANCE.NS,40.90


In [10]:
df_gap_analysis[df_gap_analysis['Stock']  == 'SIS.NS']

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Nov 2023,Stock,Growth%
122,Sales,4385.0,5833.00,7093.00,8485.00,9127.00,10059.00,11346.00,11950.0,SIS.NS,104.87
123,Profit,110.0,187.00,210.00,289.00,483.00,344.00,285.00,324.0,SIS.NS,73.26
0,Close,NaN,556.08,421.14,428.88,390.15,485.05,320.65,442.4,SIS.NS,-20.44
